# 划分训练集和测试集

## 导入工具包

In [1]:
import os
import shutil
import random
import pandas as pd

In [2]:
# 解压
!unzip dataset_full.zip >> /dev/null

## 获得所有类别名称

In [3]:
# 指定数据集路径
dataset_path = 'dataset'

In [4]:
dataset_name = dataset_path.split('_')[0]
print('数据集', dataset_name)

数据集 dataset


In [5]:
classes = os.listdir(dataset_path)

In [6]:
len(classes)

7

In [7]:
classes

['仓鼠', '兔子', '鹦鹉', '乌龟', '金鱼', '猫', '狗']

## 创建训练集文件夹和测试集文件夹

In [8]:
# 创建 train 文件夹
os.mkdir(os.path.join(dataset_path, 'train'))

# 创建 test 文件夹
os.mkdir(os.path.join(dataset_path, 'val'))

# 在 train 和 test 文件夹中创建各类别子文件夹
for fruit in classes:
    os.mkdir(os.path.join(dataset_path, 'train', fruit))
    os.mkdir(os.path.join(dataset_path, 'val', fruit))

## 划分训练集、测试集，移动文件

In [9]:
test_frac = 0.2  # 测试集比例
random.seed(123) # 随机数种子，便于复现

In [10]:
df = pd.DataFrame()

print('{:^18} {:^18} {:^18}'.format('类别', '训练集数据个数', '测试集数据个数'))

for fruit in classes: # 遍历每个类别

    # 读取该类别的所有图像文件名
    old_dir = os.path.join(dataset_path, fruit)
    images_filename = os.listdir(old_dir)
    random.shuffle(images_filename) # 随机打乱

    # 划分训练集和测试集
    testset_numer = int(len(images_filename) * test_frac) # 测试集图像个数
    testset_images = images_filename[:testset_numer]      # 获取拟移动至 test 目录的测试集图像文件名
    trainset_images = images_filename[testset_numer:]     # 获取拟移动至 train 目录的训练集图像文件名

    # 移动图像至 test 目录
    for image in testset_images:
        old_img_path = os.path.join(dataset_path, fruit, image)         # 获取原始文件路径
        new_test_path = os.path.join(dataset_path, 'val', fruit, image) # 获取 test 目录的新文件路径
        shutil.move(old_img_path, new_test_path) # 移动文件

    # 移动图像至 train 目录
    for image in trainset_images:
        old_img_path = os.path.join(dataset_path, fruit, image)           # 获取原始文件路径
        new_train_path = os.path.join(dataset_path, 'train', fruit, image) # 获取 train 目录的新文件路径
        shutil.move(old_img_path, new_train_path) # 移动文件
    
    # 删除旧文件夹
    assert len(os.listdir(old_dir)) == 0 # 确保旧文件夹中的所有图像都被移动走
    shutil.rmtree(old_dir) # 删除文件夹
    
    # 工整地输出每一类别的数据个数
    print('{:^18} {:^18} {:^18}'.format(fruit, len(trainset_images), len(testset_images)))
    
    # 保存到表格中
    df = df.append({'class':fruit, 'trainset':len(trainset_images), 'testset':len(testset_images)}, ignore_index=True)

# 重命名数据集文件夹
shutil.move(dataset_path, dataset_name+'_split')

# 数据集各类别数量统计表格，导出为 csv 文件
df['total'] = df['trainset'] + df['testset']
df.to_csv('数据量统计.csv', index=False)

        类别              训练集数据个数            测试集数据个数      
        仓鼠                348                 87        
        兔子                244                 60        
        鹦鹉                388                 97        
        乌龟                314                 78        
        金鱼                271                 67        
        猫                 365                 91        
        狗                 387                 96        


In [11]:
df

,class,trainset,testset,total
0,仓鼠,348.0,87.0,435.0
1,兔子,244.0,60.0,304.0
2,鹦鹉,388.0,97.0,485.0
3,乌龟,314.0,78.0,392.0
4,金鱼,271.0,67.0,338.0
5,猫,365.0,91.0,456.0
6,狗,387.0,96.0,483.0


## 查看文件目录结构

In [ ]:
!sudo snap install tree

In [ ]:
!tree fruit81_split -L 2